# SVA Generation Notebook

In [1]:
SPEC_NAME = 'rv'

## Assumed Preprocessed Data as follows:
- ./datasets/spec/`SPEC_NAME`.pdf: Contains the specification PDF
- ./datasets/spec/`SPEC_NAME`.csv: Contains the information of all signals in the following format: spec_name,signal_name,information. First row has signal_name = NA and contains information about signal mapping, workflow information and waveform informations.

## Output after executing this notebook:
- ./output/`SPEC_NAME`.csv: Contains the signal wise SVAs found for each signal with the following headers: spec_name,signal_name,information,output. The output contains the final reasoning process and the combined SVAs.
- ./output/json/`SPEC_NAME`_`SIGNAL_NAME`.json: Signal wise reasoning traces of MCTSr run.

## Imports

In [2]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"

In [3]:
import math
import os
import re
import json
import numpy as np
import pandas as pd
import time
from retry import retry
from config import *
from dotenv import load_dotenv
from openai import OpenAI
from utils import *

if not os.path.exists("./output"):
    os.makedirs("./output")
    os.makedirs("./output/json/")

load_dotenv()

True

In [4]:
# Function to load API keys dynamically
def load_keys():
    load_dotenv()
    return [os.environ.get(f"NEBIUS_API_KEY{i}") for i in range(1, 25) if os.environ.get(f"NEBIUS_API_KEY{i}")]

In [5]:
key = load_keys()

In [6]:
from itertools import cycle
key_cycle = cycle(key)
current_key = next(key_cycle)

## LLM Generate Command

In [7]:
CurrentSignal = 0
CurrentIteration = 0
CurrentStep = 0

In [8]:
@retry()
def generate(prompt, history=[], timeout = 50, truncate=True, model="critic"):
    time0 = time.time()
    global current_key, CurrentSignal, CurrentIteration, CurrentStep

    # Function to reload keys dynamically and preserve the current key position
    def reload_keys():
        global keys, key_cycle
        keys = load_keys()  # Reload all keys from .env
        if current_key in keys:
            key_cycle = cycle(keys)  # Restart cycle with updated keys
            while next(key_cycle) != current_key:  # Fast-forward to current key
                pass
        else:
            key_cycle = cycle(keys)  # Start fresh if key is missing

    client = OpenAI(
        # base_url = 'http://172.27.0.1:11434/v1',
        base_url="https://api.studio.nebius.ai/v1/",
        # base_url = "https://openrouter.ai/api/v1",
        # api_key = os.environ.get('OPENROUTER')
        api_key = current_key,
        # api_key='ollama', # required, but unused
    )

    if truncate:
        if(len(history)>2):
            history = [history[0]] + [history[1]] + [history[-1]]

    history_ = [{"role": "user" if i%2 == 0 else 'assistant', "content": h} for i, h in enumerate(history)]

    if model == "critic":
        system_prompt = CRITIC_SYSTEM_PROMPT
    elif model == "generator":
        system_prompt = SVA_SYSTEM_PROMPT

    messages = [{"role": "system", "content": system_prompt}] + history_ + [{"role": "user", "content": prompt}]
    
    full_response = ""  # Store the complete response as a string

    try:
        with open(f"LLMOutput/{SPEC_NAME}_{CurrentSignal}_{CurrentIteration}_{CurrentStep}.txt", "w") as text_file:
            text_file.write(f"LLM Input: {messages}\n\n\nLLM Output: ")  # Initialize the file

        # Stream the response
        stream = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            temperature=0.0,
            max_tokens=100000,
            stream=True  # Enable streaming
        )

        with open(f"LLMOutput/{SPEC_NAME}_{CurrentSignal}_{CurrentIteration}_{CurrentStep}.txt", "a") as text_file:  # Open in append mode
            for chunk in stream:
                if chunk.choices and chunk.choices[0].delta and chunk.choices[0].delta.content:
                    token = chunk.choices[0].delta.content
                    full_response += token  # Store token in memory
                    text_file.write(token)  # Write token to file
                    text_file.flush()  # Ensure immediate writing

    except Exception as e:
        error_message = str(e)
        print(f'[Error] {error_message}, retrying...')

        if "fund" in error_message.lower():
            print("[INFO] Switching API key due to fund-related error...")
            reload_keys()  # Reload keys while preserving current position
            current_key = next(key_cycle)  # Move to the next key
            raise Exception("Switched API key due to funding issue, retrying...")  # Trigger retry
        
        raise Exception(error_message)
        response = f"Exception: {e}"
        print(f'[Error] {response}, retrying...')
        raise Exception(response)

    print(f'[INFO] Response Received! Time Taken: {time.time()-time0} seconds.')

    # with open("output.txt", "w") as text_file:
    #     text_file.write(f"LLM Output: {completion.choices[0].message.content}")
    cleaned_text = re.sub(r"<think>.*?</think>", "", full_response).strip()

    return cleaned_text, list(history) + [cleaned_text]

In [9]:
pattern = re.compile(r'\-?\d+\.\d+|\-?\d+')

@retry()
def cal_reward(question,ans,history):
    # Get jaspergold feedback
    global CurrentSignal, CurrentIteration
    if CurrentIteration == 'S':
        jgFeedback, toContinue = run_jg(ans, SPEC_NAME, CurrentSignal, 'S')
    else:
        jgFeedback, toContinue = run_jg(ans, SPEC_NAME, CurrentSignal, 'Testing')
    query = f'Generated SVAs with Reasoning: {question}\nAnswer:{ans}\n. Here is the formal verification tool feedback: {jgFeedback}. Analyze this Answer Strictly and Critic, point out every flaw for every possible imperfect to minus every possible score! You need to be very harsh and mean in calculating grades, and never give full marks to ensure that the marks are authoritative. \nOutput a score between [-100,+100]. \nResponse format:\n[Analysis]...[Score]...'
    ret = generate(query, history, truncate=True, model="critic")

    score = ret[0].split('Score')[-1]
    scores = pattern.findall(score)

    if not scores:
        raise Exception('Failed to get reward from answer')
    else:
        ret = float(scores[-1])
        if ret >= 95:
            ret = 50
        return ret, toContinue


## RAG

In [10]:
# save vectorstore using faiss
VECTORSTORE_PATH = "books/faiss_index"
METADATA_PATH = "books/documents.pkl"

import os
import pickle
import faiss
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain_community.retrievers import BM25Retriever

# Function to save metadata
def save_metadata(documents, path):
    with open(path, "wb") as f:
        pickle.dump(documents, f)

# Function to load metadata
def load_metadata(path):
    with open(path, "rb") as f:
        return pickle.load(f)
    
embedding_model = OpenAIEmbeddings(openai_api_key=os.environ.get("API_KEY"))

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
file_paths = [
    "books/pallab_dasgupta__auth-_a_roadmap_for_formal_prbook4you.pdf", 
    "books/SystemVerilog Assertions and Functional Coverage ( PDFDrive ).pdf",
    "books/systemverilog-assertions.pdf",
    "books/vdoc.pub_formal-verification-an-essential-toolkit-for-modern-vlsi-design.pdf"
]

if not os.path.exists(VECTORSTORE_PATH) and os.path.exists(METADATA_PATH):
    print("Started Loading PDF Content")

    # Load documents from the local files
    docs = []
    for file_path in file_paths:
        print(f"Currently Loading: {file_path}")
        loader = UnstructuredFileLoader(file_path)
        docs.extend(loader.load())

    print("Done Loading PDF Content")

    # Initialize a text splitter with specified chunk size and overlap
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=100
    )
    # Split the documents into chunks
    doc_splits = text_splitter.split_documents(docs)

    print("Done Splitting PDF Content into chunks")

In [12]:
# Check if FAISS index exists
if os.path.exists(VECTORSTORE_PATH) and os.path.exists(METADATA_PATH):
    print("Loading existing FAISS vector store...")
    vectorstore = FAISS.load_local(VECTORSTORE_PATH, embedding_model, allow_dangerous_deserialization=True)
    doc_splits = load_metadata(METADATA_PATH)
else:
    print("Creating new vector store...")
    vectorstore = FAISS.from_documents(doc_splits, embedding_model)
    vectorstore.save_local(VECTORSTORE_PATH)
    save_metadata(doc_splits, METADATA_PATH)
    print("Vector store saved for future use.")

Loading existing FAISS vector store...


In [13]:
# Create retriever
retriever = vectorstore.as_retriever(k=10)

# Define RAG application
class RAGApplication:
    def __init__(self, retriever):
        self.retriever = retriever

    def run(self, question):
        documents = self.retriever.invoke(question)
        doc_texts = "\n".join([doc.page_content for doc in documents])
        return doc_texts

rag_application = RAGApplication(retriever)

## MCTS Initialisation Functions

In [14]:
@retry()
def get_weak_answer(question,history):
    query = f'{question}\nThe response should begin with [Reasoning Process]... [Verification]... and end with [Answer]... \nLet\'s think step by step.'
    return generate(query, history, model = "generator")

def get_weak_hints(question,weak_answer,history=[]):
    query = f'{question}.\nSince we have a weak Answer from the specifications: {weak_answer}, could you provide me with a reflection or feedback to correct this answer better? Analyze this Answer Strictly and Critic, point out every flaw for ervery possible imperfect to minus every possible score!\nLet\'s think step by step.'
    return generate(query, history, model="critic")

def get_better_answer(question,weak_answer,hint,jg_feedback,history=[]):
    query = f'{question}.\nPlease refine your old answer: {weak_answer}.\n\nAccording to the Feedback: {hint}. Here are the syntax error report for each signal: {jg_feedback}. The response should begin with [reasoning process]...[Verification]... and end with end with [Answer]\nLet\'s think step by step.'

    # context = getContextFromBooks2(query, books, 10)
    context = rag_application.run(question)
    query += "Additional Context: " + context

    query += "\nPlease segregate the output assertions in the following format: ```systemverilog [width] ... [connectivity] ... [functionality] ... ```. Dont hallucinate new information if it is not present in the specification."

    return generate(query, history, model="generator")

## MCTS Runtime Functions

In [15]:
datas = []

def sort_answers_and_rewards(answers, rewards):
    # Zip answers and rewards together
    answer_reward_pairs = zip(answers, rewards)
    
    # Sort pairs by rewards
    sorted_pairs = sorted(answer_reward_pairs, key=lambda x: x[1], reverse=True)
    
    # Extract sorted answers and rewards
    sorted_answers = [pair[0] for pair in sorted_pairs]
    sorted_rewards = [pair[1] for pair in sorted_pairs]
    
    return sorted_answers, sorted_rewards

def filter_mature_node(childs, to_explore, to_explore_reward, max_expand=3):
    filterd_to_explore = []
    avg_reward = {node: (min(to_explore_reward[node]) + np.mean(to_explore_reward[node])) / 2 for node in to_explore}

    for node in to_explore:
        if len(childs.get(node,[])) < max_expand or max([avg_reward.get(child,-999) for child in childs.get(node,[])]) < avg_reward.get(node,-999):
            filterd_to_explore.append(node)
    
    return filterd_to_explore

def get_best_explore_from_ucb(to_explore, ucb_bank):
    best_node = None
    highest_ucb = float('-inf')
    
    for node in to_explore:
        ucb_value = ucb_bank.get(node, float('-inf'))
        if ucb_value > highest_ucb:
            highest_ucb = ucb_value
            best_node = node
            
    return best_node


In [16]:
def compute_ucb(r_c, N_n, N_c, C):
    return r_c + C * math.sqrt(math.log(N_n + 1) / (N_c + 1e-5))

def update_ucb(fathers, childs, to_explore, to_explore_reward, ucb_bank, C=1.4,gamma=0.85):
    visit_count = {node: len(to_explore_reward[node]) for node in to_explore}

    avg_reward = {node: (min(to_explore_reward[node]) + np.mean(to_explore_reward[node])) / 2 for node in to_explore}

    leaves = set(to_explore) - set(fathers.values())
    
    for leaf in leaves:
        # ucb_bank[leaf] = avg_reward[leaf]
        ucb_bank[leaf] = compute_ucb(avg_reward[leaf],len(to_explore_reward.get(fathers.get(leaf,None),[])),len(to_explore_reward.get(leaf,[])),C)
    
    nodes_to_update = list(leaves)
    while nodes_to_update:
        new_nodes_to_update = set()
        for node in nodes_to_update:
            father = fathers.get(node)
            if father is not None:
                if father not in ucb_bank:
                    new_nodes_to_update.add(father)
                if father in ucb_bank:
                    ucb_values = []
                    child_reward = []
                    for child in childs[father]:
                        ucb_values.append(ucb_bank[child])
                        child_reward.append(avg_reward[child])
                    father_reward = (avg_reward[father] + max(child_reward))/2
                    ucb_bank[father] = compute_ucb(father_reward,len(to_explore_reward.get(fathers.get(father,None),[])),len(to_explore_reward.get(father,[])),C)
        nodes_to_update = list(new_nodes_to_update)

weak_answer_test = ""

def step(query, weak_answer, history=[], signal_name = "", i = 0):
    global CurrentStep
    print("[MCTS] Generating Feedback from Node")
    CurrentStep = 'GetHints'
    hints,history = get_weak_hints(query,weak_answer,history=history)

    # get feedback from jg
    global weak_answer_test
    weak_answer_test = weak_answer
    jg_feedback, toContinue = run_jg(weak_answer, SPEC_NAME, signal_name, i)

    print("[MCTS] Generating a better set of SVAs")
    CurrentStep = 'GetBetterAnswer'
    answer,history = get_better_answer(query,weak_answer,hints,jg_feedback,history=history)
    return hints,answer,history, toContinue

## Main MCTS Loop

In [17]:
def main_loop(query,max_iter=4, historyOG=[], signal_name = ""):
    global CurrentSignal, CurrentIteration, CurrentStep
    CurrentSignal = signal_name

    to_explore = []
    to_explore_reward = {}
    history_bank = {}
    hints_bank = {}
    ucb_bank = {}
    fathers = {}
    childs = {}
    def sampling_reward(answer, history):
        if answer not in to_explore_reward:
            to_explore_reward[answer] = []
        reward, toContinue = cal_reward(query,answer,history)
        print(f"Got Reward: {reward}")
        to_explore_reward[answer].append(reward)
        return toContinue

    def add_to_hints_bank(hints,weak_answer):
        if weak_answer not in hints_bank:
            hints_bank[weak_answer] = []
        hints_bank[weak_answer].append(hints)

    def add_to_childs(father,child):
        if father not in childs:
            childs[father] = []
        childs[father].append(child)

    hints_reward_imp_bank = {}
    def add_to_hints_reward_imp_bank(hints,weak_answer,reward,answer):
        if weak_answer not in hints_reward_imp_bank:
            hints_reward_imp_bank[weak_answer] = []
        hints_reward_imp_bank[weak_answer].append((hints,reward,answer))

    ###get weak answer###
    CurrentIteration = 'S'
    CurrentStep = 'WeakAnswer'
    print("[MCTS] Generating Weak Answer Node")
    weak_answer, history = get_weak_answer(query, historyOG[0])
    history_bank[weak_answer] = tuple(history)
    answers_list = [weak_answer,]
    to_explore = [weak_answer,]
    childs[weak_answer] = []
    fathers[weak_answer] = None
    print("[MCTS] Sampling reward for weak answer")
    CurrentStep = 'SampleReward'
    toContinue = sampling_reward(weak_answer, historyOG[0])

    hints_list = []

    update_ucb(fathers=fathers, childs=childs, to_explore=to_explore, to_explore_reward=to_explore_reward, ucb_bank=ucb_bank)

    for i in range(max_iter):
        CurrentIteration = i
        if toContinue == True:
            pass
        else:
            print("Already obtained completely correct set of assertions. Breaking")
            break

        # Print the rollout number
        print('[INFO] Iteration:', i)

        # Node Selection
        print("[MCTS] Step 1: Selecting Best Node and Resampling Reward")
        filterd_to_explore = filter_mature_node(childs, to_explore, to_explore_reward)
        # Update reward calculated for node again
        weak_answer = get_best_explore_from_ucb(filterd_to_explore, ucb_bank)
        CurrentStep = 'ResampleReward'
        sampling_reward(weak_answer, historyOG[0])

        # Node Expansion
        print("[MCTS] Step 2: Expanding Node")
        hints,answer,history,toContinue = step(query,weak_answer,history=history_bank[weak_answer], signal_name = signal_name, i = i)
        add_to_hints_bank(hints,weak_answer)
        history_bank[answer] = tuple(history)
        to_explore.append(answer)

        if toContinue == True:
            pass
        else:
            print("Already obtained completely correct set of assertions. Breaking")
            break

        # Node Evaluation
        print("[MCTS] Step 3: Node Evaluation")
        CurrentStep = 'EvaluateNewNode'
        sampling_reward(answer, historyOG[0])

        # Auxiliary functions for updating tree
        fathers[answer] = weak_answer
        childs[answer] = []
        add_to_childs(weak_answer,answer)
        answers_list.append(answer)
        hints_list.append(hints)

        # Backpropogation
        print("[MCTS] Step 4: Backpropogating through the tree\n")
        update_ucb(fathers=fathers,childs=childs,to_explore=to_explore,to_explore_reward=to_explore_reward,ucb_bank=ucb_bank)
        add_to_hints_reward_imp_bank(hints,weak_answer,min(to_explore_reward.get(answer)) - min(to_explore_reward.get(weak_answer)),answer)#ucb_bank[answer] - ucb_bank[weak_answer]

    return hints_list,answers_list,to_explore,to_explore_reward,hints_bank,history_bank,hints_reward_imp_bank,fathers,childs,ucb_bank

## Caller Function

In [18]:
from time import sleep

In [19]:
def func(signal_name, signal_information, history, spec_name, document_data):
    query = "Generate SVAs for " + signal_name + ". " + signal_information
    max_iter = 4

    hints_list, answers_list, to_explore, to_explore_reward, hints_bank, history_bank, hints_reward_imp_bank, fathers, childs,ucb_bank = main_loop(query, max_iter, history, signal_name)

    data = {
        'query':query,
        'hints_list':hints_list,
        'answers_list':answers_list,
        'to_explore':to_explore,
        'to_explore_reward':to_explore_reward,
        'hints_bank':hints_bank,
        'history_bank':history_bank,
        'hints_reward_imp_bank':hints_reward_imp_bank,
        'fathers':fathers,
        'childs':childs,
        'ucb_bank':ucb_bank,
    }

    with open(f'./output/json/{spec_name}_{signal_name}.json','w+') as f:
        json.dump(data,f,indent=4,ensure_ascii=False)

## Execution Code

In [21]:
directory = "./datasets/info/"
pdf, spec_name = SPEC_NAME + '.pdf', SPEC_NAME
document_data = read_document("./datasets/spec/" + pdf)

In [36]:
df = pd.read_csv("./datasets/info/" + spec_name + '.csv', quotechar="'")

In [37]:
df

,spec_name,signal_name,information
0,rv,NaN,Here is the mapping information:\r\n[clk_i]: 1...
1,rv,clk_i,\r\n[Signal name]: clk_i\r\n[Description]:\r\n...
2,rv,rst_ni,\r\n[Signal name]: rst_ni\r\n[Description]:\r\...
3,rv,active,\r\n[Signal name]: active \r\n[Description]: ...
4,rv,prescaler,\r\n[Signal name]: prescaler \r\n[Description...
5,rv,step,\r\n[Signal name]: step \r\n[Description]: \...
6,rv,mtime,\r\n[Signal name]: mtime \r\n[Description]: ...
7,rv,mtimecmp,\r\n[Signal name]: mtimecmp \r\n[Description]...
8,rv,tick,\r\n[Signal name]: tick \r\n[Description]: \...
9,rv,mtime_d,\r\n[Signal name]: mtime_d \r\n[Description]:...


In [38]:
history = []
history.append([document_data + df.loc[0, 'information']])

In [39]:
def processRow(rowindex, allSteps = True, history = [], justCombine = False):
    global document_data
    
    for index, row in df.iterrows():
        if index == 0:
            continue

        if index != rowindex:
            continue

        signal_name = row['signal_name']
        signal_information = row['information']
        spec_name = row['spec_name'] 
        document_data = document_data

        if allSteps:
            func(signal_name, signal_information, history, spec_name, document_data) # This creates the json

        if justCombine:
            try:
                assertions = getValidAssertions(SPEC_NAME, signal_name, document_data + df.loc[0]['information'] + signal_information, "https://api.studio.nebius.ai/v1/", current_key)

                with open(f"{signal_name}.txt", 'w') as file:
                    file.write(assertions)

                df.loc[index:index, 'COMBINER'] = assertions
            except Exception as e:
                print("Modified Combiner Failed", e)

            try:
                # Write Dataframe
                df.to_csv(f"TempOutput/{signal_name}.csv", index=False)
            except:
                print("Dataframe Write to Disk Failed")

            return

        try:
            print(f"Starting Combiner step for {signal_name}")

            # Open and read the JSON file
            with open(f'./output/json/{spec_name}_{signal_name}.json', 'r') as f:
                data = json.load(f)

            # Extract 'query' and 'history_bank' into a new dictionary
            data = {
                'query': data.get('query'),
                'history_bank': data.get('history_bank')
            }    

            # Modify the dictionary
            first_key = next(iter(data['history_bank']))

            for key in data['history_bank']:
                if key != first_key:  # Skip the first key
                    data['history_bank'][key] = data['history_bank'][key][1:]  # Remove the first element

            client = OpenAI(
                # base_url = 'http://172.27.0.1:11434/v1',
                # base_url = "https://openrouter.ai/api/v1",
                # api_key = os.environ.get('OPENROUTER')
                base_url="https://api.studio.nebius.ai/v1/",
                api_key=current_key,
            )

            full_response = ""

            # Stream the response
            stream = client.chat.completions.create(
                model=MODEL_NAME,
                messages = [
                    {"role": "system", "content": COMBINER_PROMPT},
                    {"role": "user", "content": f"Combine all this reasoning data {data} along with your own reasoning, and produce a set of COMPLETE, CONSISTENT and CORRECT SVAs. Here is the signal name {signal_name}."}
                ],
                temperature=0.0,
                max_tokens=128000,
                stream=True  # Enable streaming
            )

            with open("output.txt", "w") as text_file:  # Open in append mode
                for chunk in stream:
                    if chunk.choices and chunk.choices[0].delta and chunk.choices[0].delta.content:
                        token = chunk.choices[0].delta.content
                        full_response += token  # Store token in memory
                        text_file.write(token)  # Write token to file
                        text_file.flush()  # Ensure immediate writing

            try:
                with open(f'./LLMOutput/{signal_name}.txt', "w") as file:
                    file.write(full_response)
            except:
                print("Write to file failed")

            try:
                # run jg with final values
                if(full_response != ""):
                    run_jg(full_response, SPEC_NAME, signal_name, 'FINAL')
                else:
                    print("Failed to get Feedback for Combined LLM: Response Empty")
            except:
                print("Failed to get Feedback for Combined LLM")

            try:
                df.loc[index:index, 'output'] = full_response
            except:
                print("Save to df failed")


            try:
                assertions = getValidAssertions(SPEC_NAME, signal_name, document_data + df.loc[0:0, 'information'].to_string()[5:], "https://api.studio.nebius.ai/v1/", current_key)

                with open(f"{signal_name}.txt", 'w') as file:
                    file.write(assertions)

                df.loc[index:index, 'COMBINER'] = assertions
            except:
                print("Modified Combiner Failed")

            try:
                # Write Dataframe
                df.to_csv(f"TempOutput/{signal_name}.csv", index=False)
            except:
                print("Dataframe Write to Disk Failed")


            print(f"Completed Combiner step for {signal_name}")

        except Exception as e:
            print(f"Failed last step for {signal_name} {e}")


In [40]:
processRow(1, True, history)

[MCTS] Generating Weak Answer Node
[Error] Error code: 402 - {'detail': 'Payment Required: You have exhausted your budget. Please add funds to continue using the API.'}, retrying...
[INFO] Switching API key due to fund-related error...
[Error] Error code: 402 - {'detail': 'Payment Required: You have exhausted your budget. Please add funds to continue using the API.'}, retrying...
[INFO] Switching API key due to fund-related error...
[Error] peer closed connection without sending complete message body (incomplete chunked read), retrying...
[INFO] Response Received! Time Taken: 476.0045783519745 seconds.
[MCTS] Sampling reward for weak answer
/storage/ckarfa/hpdmc/sva/rv/rv.sv
assert property (@(posedge clk_i) $bits(clk_i) == 1);
--------------------------------------------------------------------------------
assert property (@(posedge clk_i) $stable(tick) || $changed(tick));
--------------------------------------------------------------------------------
property MTIME_INCREMENT_ON_TICK

In [41]:
processRow(2, True, history)

[MCTS] Generating Weak Answer Node
[INFO] Response Received! Time Taken: 759.9985644817352 seconds.
[MCTS] Sampling reward for weak answer
/storage/ckarfa/hpdmc/sva/rv/rv.sv
assert property (@(posedge clk_i) ($bits(rst_ni) == 1));
--------------------------------------------------------------------------------
property rst_ni_intr_connectivity;
  @(posedge clk_i) !rst_ni |-> (intr == 0);
endproperty
assert property (rst_ni_intr_connectivity);
--------------------------------------------------------------------------------
property rst_ni_mtime_connectivity;
  @(posedge clk_i) !rst_ni |-> (mtime == 64'h0);
endproperty
assert property (rst_ni_mtime_connectivity);
--------------------------------------------------------------------------------
property rst_ni_tick_connectivity;
  @(posedge clk_i) !rst_ni |-> !tick;
endproperty
assert property (rst_ni_tick_connectivity);
--------------------------------------------------------------------------------
property rst_ni_async_reset;
  @(posedg

In [42]:
processRow(3, True, history)

[MCTS] Generating Weak Answer Node
[INFO] Response Received! Time Taken: 228.31527161598206 seconds.
[MCTS] Sampling reward for weak answer
/storage/ckarfa/hpdmc/sva/rv/rv.sv
assert property (@(posedge clk_i) disable iff (!rst_ni) ($bits(active) == 1));
--------------------------------------------------------------------------------
property active_inactive_mtime_stable;
  @(posedge clk_i) disable iff (!rst_ni)
  !active |-> (mtime_d == mtime);
endproperty
assert property (active_inactive_mtime_stable);
--------------------------------------------------------------------------------
property active_inactive_no_tick;
  @(posedge clk_i) disable iff (!rst_ni)
  !active |-> !tick;
endproperty
assert property (active_inactive_no_tick);
--------------------------------------------------------------------------------
property active_inactive_no_intr;
  @(posedge clk_i) disable iff (!rst_ni)
  !active |-> !intr[0];
endproperty
assert property (active_inactive_no_intr);
------------------------

In [43]:
processRow(4, True, history)

[MCTS] Generating Weak Answer Node
[INFO] Response Received! Time Taken: 449.3033504486084 seconds.
[MCTS] Sampling reward for weak answer
/storage/ckarfa/hpdmc/sva/rv/rv.sv
assert property (@(posedge clk_i) $bits(prescaler) == 12);
--------------------------------------------------------------------------------
property prescaler_tick_period;
  int p;
  @(posedge clk_i) disable iff (!rst_ni || !active)
  (tick, p = prescaler) |-> ##(p +1) tick;
endproperty
assert property (prescaler_tick_period);
--------------------------------------------------------------------------------
property no_tick_inactive;
  @(posedge clk_i) disable iff (!rst_ni)
  !active |-> !tick;
endproperty
assert property (no_tick_inactive);
--------------------------------------------------------------------------------
property first_tick_post_activation;
  @(posedge clk_i)
  $rose(active) && rst_ni |-> ##(prescaler +1) tick;
endproperty
assert property (first_tick_post_activation);
-------------------------------

In [44]:
processRow(5, True, history)

[MCTS] Generating Weak Answer Node
[INFO] Response Received! Time Taken: 341.91448760032654 seconds.
[MCTS] Sampling reward for weak answer
/storage/ckarfa/hpdmc/sva/rv/rv.sv
assert property (@(posedge clk_i) ($bits(step) == 8));
--------------------------------------------------------------------------------
property step_connectivity;
  @(posedge clk_i) disable iff (!rst_ni)
  (active && tick) |-> (mtime_d == (mtime + step));
endproperty
assert property (step_connectivity);
--------------------------------------------------------------------------------
property step_functionality_inactive;
  @(posedge clk_i) disable iff (!rst_ni)
  (!active) |-> (mtime_d == mtime);
endproperty
assert property (step_functionality_inactive);
--------------------------------------------------------------------------------
Getting Feedback from JG - Starting File Upload
Getting Feedback from JG - File Upload Successful
Getting Feedback from JG - Command Execution Successful. Downloading Logs
Getting Fee

In [45]:
processRow(6, True, history)

[MCTS] Generating Weak Answer Node
[INFO] Response Received! Time Taken: 394.6009306907654 seconds.
[MCTS] Sampling reward for weak answer
/storage/ckarfa/hpdmc/sva/rv/rv.sv
assert property (@(posedge clk_i) $bits(mtime) == 64);
--------------------------------------------------------------------------------
property mtime_increment_p;
  @(posedge clk_i) disable iff (!rst_ni)
  (active && tick) |-> (mtime_d == (mtime + step));
endproperty
assert property (mtime_increment_p);
--------------------------------------------------------------------------------
property mtime_hold_p;
  @(posedge clk_i) disable iff (!rst_ni)
  (active && !tick) |-> (mtime_d == mtime);
endproperty
assert property (mtime_hold_p);
--------------------------------------------------------------------------------
property mtime_intr_p;
  @(posedge clk_i) disable iff (!rst_ni)
  (mtime >= mtimecmp[0]) |-> intr[0];
endproperty
assert property (mtime_intr_p);
------------------------------------------------------------

In [46]:
processRow(7, True, history)

[MCTS] Generating Weak Answer Node
[INFO] Response Received! Time Taken: 222.5825753211975 seconds.
[MCTS] Sampling reward for weak answer
/storage/ckarfa/hpdmc/sva/rv/rv.sv
property mtimecmp_intr_p;
  @(posedge clk_i) disable iff (!rst_ni || !active)
    (mtime >= mtimecmp[0]) |-> intr[0];
endproperty
assert property (mtimecmp_intr_p);
--------------------------------------------------------------------------------
property mtimecmp_no_intr_p;
  @(posedge clk_i) disable iff (!rst_ni || !active)
    (mtime < mtimecmp[0]) |-> !intr[0];
endproperty
assert property (mtimecmp_no_intr_p);
--------------------------------------------------------------------------------
property mtimecmp_stable_p;
  @(posedge clk_i) disable iff (!rst_ni)
    active |-> $stable(mtimecmp);
endproperty
assert property (mtimecmp_stable_p);
--------------------------------------------------------------------------------
Getting Feedback from JG - Starting File Upload
Getting Feedback from JG - File Upload Successf

In [47]:
processRow(8, True, history)

[MCTS] Generating Weak Answer Node
[INFO] Response Received! Time Taken: 409.5001494884491 seconds.
[MCTS] Sampling reward for weak answer
/storage/ckarfa/hpdmc/sva/rv/rv.sv
assert property (@(posedge clk_i) $bits(tick) == 1);
--------------------------------------------------------------------------------
property tick_reset_inactive;
  @(posedge clk_i) !rst_ni || !active |-> !tick;
endproperty
assert property (tick_reset_inactive);
--------------------------------------------------------------------------------
property tick_single_pulse;
  @(posedge clk_i) disable iff (!rst_ni || !active)
  tick |=> (prescaler == 0) || !tick;
endproperty
assert property (tick_single_pulse);
--------------------------------------------------------------------------------
property mtime_update_on_tick;
  @(posedge clk_i) disable iff (!rst_ni)
  tick |-> mtime_d == mtime + step;
endproperty
assert property (mtime_update_on_tick);
-------------------------------------------------------------------------

In [48]:
processRow(9, True, history)

[MCTS] Generating Weak Answer Node
[INFO] Response Received! Time Taken: 287.500759601593 seconds.
[MCTS] Sampling reward for weak answer
/storage/ckarfa/hpdmc/sva/rv/rv.sv
assert property (@(posedge clk_i) $bits(mtime_d) == 64);
--------------------------------------------------------------------------------
property mtime_d_connectivity_tick;
  @(posedge clk_i)
  tick |-> (mtime_d == (mtime + step));
endproperty
assert property (mtime_d_connectivity_tick);
--------------------------------------------------------------------------------
property mtime_d_connectivity_no_tick;
  @(posedge clk_i)
  !tick |-> (mtime_d == mtime);
endproperty
assert property (mtime_d_connectivity_no_tick);
--------------------------------------------------------------------------------
property mtime_update;
  @(posedge clk_i)
  mtime == $past(mtime_d, 1);
endproperty
assert property (mtime_update);
--------------------------------------------------------------------------------
property mtime_d_stability;


In [49]:
processRow(10, True, history)

[MCTS] Generating Weak Answer Node
[INFO] Response Received! Time Taken: 519.6531095504761 seconds.
[MCTS] Sampling reward for weak answer
/storage/ckarfa/hpdmc/sva/rv/rv.sv
assert property (@(posedge clk_i) $bits(intr[i]) == 1);
--------------------------------------------------------------------------------
property intr_reset_p;
  @(posedge clk_i) !rst_ni |-> !intr;
endproperty
assert property (intr_reset_p);
--------------------------------------------------------------------------------
property intr_compare_p;
    @(posedge clk_i) rst_ni |-> (intr[i] == (mtime >= mtimecmp[i]));
  endproperty
  assert property (intr_compare_p);
--------------------------------------------------------------------------------
Getting Feedback from JG - Starting File Upload
Getting Feedback from JG - File Upload Successful
Getting Feedback from JG - Command Execution Successful. Downloading Logs
Getting Feedback from JG - Starting File Upload
Getting Feedback from JG - File Upload Successful
Getting 

In [50]:
df.to_csv('rvFinal.csv', index=False)